In [2]:
import json
import pandas as pd
import requests
from tqdm import tqdm

In [4]:
un_db = pd.read_csv("top_10k_unigr.csv")
un_db.head()

,word_id,ref_id,setting_id,translate
0,44776,3,1,vigia
1,14498,3,1,empresário
2,6623,3,1,taxista
3,35321,3,1,vendedor
4,10022,3,1,conselheiro


In [6]:
import spacy

In [7]:
nlp = spacy.load("/Users/lilyakhoang/input/pt_core_news_sm-2.1.0/pt_core_news_sm/pt_core_news_sm-2.1.0")

In [19]:
text = ""
pos = ""
lemma = ""
for token in nlp("Estás bem ?"):
    print(token.lemma_)
    text += token.text
    pos += token.pos_ + " "
    lemma += token.lemma_ + " "

Estás
bem
?


In [26]:
def generate_letter_count_dict_port(unigraamm_db):

    popular_unigramms = list(unigraamm_db['translate'])
    letter_count_dict = {}
    for word_ind in tqdm(range(len(popular_unigramms))):
        word = popular_unigramms[word_ind]
        try:
            tr_id = unigraamm_db.iloc[word_ind]['setting_id']
            word_id = unigraamm_db.iloc[word_ind]['word_id']
            ref_id = unigraamm_db.iloc[word_ind]['ref_id']
            """analysis"""
            for token in nlp(word):
                pos = token.pos_
                lemma = token.lemma_
                break

            #print(word,pos, lemma)
            #break

            #print(lemma, pos)

            if len(word) in letter_count_dict:
                letter_count_dict[len(word)].append((word.lower(),pos,lemma,int(word_id),int(ref_id),int(tr_id)))
            else:
                letter_count_dict[len(word)] = []
                letter_count_dict[len(word)].append((word.lower(),pos,lemma,int(word_id),int(ref_id),int(tr_id)))
        except:
            print("failed on", word)
    return  letter_count_dict
let_count_d = generate_letter_count_dict_port(un_db)


 10%|█         | 1034/10000 [00:18<02:37, 56.89it/s]


 22%|██▏       | 2225/10000 [00:35<02:02, 63.54it/s]


 32%|███▏      | 3177/10000 [00:53<01:54, 59.48it/s]


 42%|████▏     | 4156/10000 [01:11<01:40, 58.08it/s]


 53%|█████▎    | 5265/10000 [01:28<01:19, 59.50it/s]


...

 95%|█████████▌| 9538/10000 [02:38<00:07, 60.01it/s]

failed on nan



 99%|█████████▉| 9939/10000 [02:45<00:01, 60.16it/s]

failed on nan



100%|██████████| 10000/10000 [02:46<00:00, 60.19it/s]

In [27]:
with open("unigram_pos_lemma_por.json","w") as f:
    json.dump(let_count_d, f, indent = 1, ensure_ascii=False)

In [20]:
from TurkishStemmer import TurkishStemmer
stemmer = TurkishStemmer()
import requests

def generate_letter_count_dict_turk(unigraamm_db):
    headers_tr = {
    'cache-control': 'no-cache',
    'content-type': 'application/json',
    'postman-token': 'c18af364-c1cb-cc41-0903-063547ac7fce',
    }
    url_tr = 'http://localhost:8081/analyze'
    
    popular_unigramms = list(unigraamm_db['translate'])
    letter_count_dict = {}
    for word_ind in tqdm(range(len(popular_unigramms))):
        word = popular_unigramms[word_ind]
        tr_id = unigraamm_db.iloc[word_ind]['setting_id']
        word_id = unigraamm_db.iloc[word_ind]['word_id']
        ref_id = unigraamm_db.iloc[word_ind]['ref_id']
        """analysis"""
        data = {"tokens" : [word]}
        data_dump = json.dumps(data)
        r = requests.post(url = url_tr, data=data_dump, headers=headers_tr)
        lemma = stemmer.stem(word)
        pos = r.text.split("+")[1].lower()
        
        #print(lemma, pos)
        
        if len(word) in letter_count_dict:
            letter_count_dict[len(word)].append((word.lower(),pos,lemma,int(word_id),int(ref_id),int(tr_id)))
        else:
            letter_count_dict[len(word)] = []
            letter_count_dict[len(word)].append((word.lower(),pos,lemma,int(word_id),int(ref_id),int(tr_id)))
    return  letter_count_dict
let_count_d = generate_letter_count_dict_turk(un_db)


100%|██████████| 10000/10000 [03:03<00:00, 54.47it/s]


In [21]:
let_count_d[5]

[('zorlu', 'adj"]', 'zorl', 58436, 2, 1),
 ('olmak', 'verb', 'olmak', 4642, 2, 1),
 ('olmak', 'verb', 'olmak', 4716, 2, 1),
 ('ahşap', 'adj"]', 'ahşap', 45645, 2, 1),
 ('dosya', 'noun', 'dosya', 16562, 2, 1),
 ('gelir', 'noun', 'Gelir', 21068, 2, 1),
 ('almak', 'verb', 'almak', 40307, 2, 1),
 ('iştah', 'noun', 'iştah', 3251, 2, 1),
 ('yakın', 'noun', 'yak', 26903, 2, 1),
 ('tavan', 'noun', 'tavan', 7411, 2, 1),
 ('uçmak', 'verb', 'uçmak', 16514, 2, 1),
 ('sıcak', 'adj"]', 'sıcak', 20110, 2, 1),
 ('açmak', 'verb', 'açmak', 28239, 2, 1),
 ('güçlü', 'noun', 'güçl', 39331, 2, 1),
 ('ortak', 'noun', 'ortak', 8954, 2, 1),
 ('nabız', 'noun', 'nap', 32641, 2, 1),
 ('öksüz', 'noun', 'öks', 28453, 2, 1),
 ('küçük', 'adj"]', 'küçük', 37627, 2, 1),
 ('geniş', 'adj"]', 'geniş', 45356, 2, 1),
 ('çocuk', 'noun', 'çocuk', 7824, 2, 1),
 ('torba', 'noun', 'torp', 4182, 2, 1),
 ('silgi', 'noun', 'silg', 35034, 2, 1),
 ('fırça', 'noun', 'fırç', 6290, 2, 1),
 ('doğru', 'adj"]', 'doğr', 9877, 2, 1),
 ('duva

In [22]:
with open("unigram_pos_lemma_tur.json","w") as f:
    json.dump(let_count_d, f, indent = 1, ensure_ascii=False)

In [ ]:
RUS
def generate_letter_count_dict_turk(unigraamm_db):
    popular_unigramms = list(unigraamm_db['translate'])
    letter_count_dict = {}
    for word_ind in tqdm(range(len(popular_unigramms))):
        word = popular_unigramms[word_ind]
        tr_id = unigraamm_db.iloc[word_ind]['translation_id']
        word_id = unigraamm_db.iloc[word_ind]['word_id']
        
        """analysis"""
        p = morph.parse(word)[0]
        lemma = p.normal_form
        pos = p.tag.POS 

        if len(word) in letter_count_dict:
            letter_count_dict[len(word)].append((word.lower(),pos,lemma,int(word_id),int(tr_id)))
        else:
            letter_count_dict[len(word)] = []
            letter_count_dict[len(word)].append((word.lower(),pos,lemma,int(word_id),int(tr_id)))

In [ ]:
EN
def generate_letter_count_dict_turk(unigraamm_db):
    popular_unigramms = list(unigraamm_db['translate'])
    letter_count_dict = {}
    for word_ind in tqdm(range(len(popular_unigramms))):
        word = popular_unigramms[word_ind]
        tr_id = unigraamm_db.iloc[word_ind]['translation_id']
        word_id = unigraamm_db.iloc[word_ind]['word_id']
        p = morph.parse(word)[0]
        lemma = p.normal_form
        pos = p.tag.POS 
        
        """
        tkn = nltk.word_tokenize(word)
        word_pos = nltk.pos_tag(tkn)[0]
        wordnet_pos = get_wordnet_pos(word_pos[1]) 
        if (wordnet_pos):
            lemma = lemmatizer.lemmatize(word, pos = wordnet_pos)
        else:
            lemma = lemmatizer.lemmatize(word)
        """
        if len(word) in letter_count_dict:
            letter_count_dict[len(word)].append((word.lower(),pos,lemma,int(word_id),int(tr_id)))
        else:
            letter_count_dict[len(word)] = []
            letter_count_dict[len(word)].append((word.lower(),pos,lemma,int(word_id),int(tr_id)))